Importing Libraries


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import cv2

Checking Data

In [3]:
print("The number of images with facemask labelled 'yes':",len(os.listdir('/content/drive/My Drive/Face Mask Detection/dataset/with mask')))
print("The number of images with facemask labelled 'no':",len(os.listdir('/content/drive/My Drive/Face Mask Detection/dataset/without mask')))


The number of images with facemask labelled 'yes': 690
The number of images with facemask labelled 'no': 686


#Data Preprocessing


**Creating Directories for traininig and testing**

In [5]:
dir =['/content/drive/My Drive/Face_Mask_normalized','/content/drive/My Drive/Face_Mask','/content/drive/My Drive/Face_Mask/training', '/content/drive/My Drive/Face_Mask/testing',
      '/content/drive/My Drive/Face_Mask/training/with_mask','/content/drive/My Drive/Face_Mask/training/without_mask',
      '/content/drive/My Drive/Face_Mask/testing/with_mask','/content/drive/My Drive/Face_Mask/testing/without_mask',
      '/content/drive/My Drive/Face_Mask_normalized/with_mask','/content/drive/My Drive/Face_Mask_normalized/without_mask']
try :
  for directories in dir :
    os.mkdir(directories)
    print("Your %s has been created"%directories)
except OSError :
  pass 

In [6]:
ls = ['/content/drive/My Drive/Face_Mask_normalized/with_mask','/content/drive/My Drive/Face_Mask_normalized/without_mask']
try :
  for directories in ls :
    os.mkdir(directories)
    print("Your %s has been created"%directories)
except OSError :
  pass 

Your /content/drive/My Drive/Face_Mask_normalized/with_mask has been created
Your /content/drive/My Drive/Face_Mask_normalized/without_mask has been created


**Splitting Data into respective directories**

In [11]:
from tqdm import tqdm 

In [14]:
def normalize(source_dir,copy_dir) :
  for filename in tqdm(os.listdir(source_dir)):  
            try:
                img_array = cv2.imread(os.path.join(source_dir,filename) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (150, 150))  # resize to normalize data size
                os.chdir(copy_dir)
                cv2.imwrite(filename,new_array)
            except Exception as e:  # in the interest in keeping the output clean...
                pass

In [16]:
source_mask ='/content/drive/My Drive/Face Mask Detection/dataset/with mask/'
source_mask_normalized = '/content/drive/My Drive/Face_Mask_normalized/with_mask/'

source_without_mask ="/content/drive/My Drive/Face Mask Detection/dataset/without mask/"
source_without_mask_normalized ="/content/drive/My Drive/Face_Mask_normalized/without_mask/"

normalize(source_mask,source_mask_normalized)
normalize(source_without_mask,source_without_mask_normalized)



100%|██████████| 686/686 [00:13<00:00, 51.93it/s]


In [28]:
for data in os.listdir('/content/drive/My Drive/Face_Mask_normalized/with_mask/') :
  img = cv2.imread('/content/drive/My Drive/Face_Mask_normalized/with_mask/'+data)
  print(img.shape)
  break
  

(150, 150, 3)


In [17]:
def split_files(source_dir,training_dir,testing_dir,split_size):
  repo =[]
  for filenames in tqdm(os.listdir(source_dir) ):
    if os.path.getsize(source_dir+filenames):
      repo.append(filenames)

  length = len(repo)
  split = int(split_size*length)
  random.sample(repo,length)
  training =repo[:split]
  testing = repo[split:]
  for files in training :
    copyfile(source_dir+files,training_dir+files)
  for files in testing :
    copyfile(source_dir+files,testing_dir+files)


In [18]:
training_with_mask = "/content/drive/My Drive/Face_Mask/training/with_mask/"
training_without_mask ="/content/drive/My Drive/Face_Mask/training/without_mask/"
testing_with_mask = "/content/drive/My Drive/Face_Mask/testing/with_mask/"
testing_without_mask="/content/drive/My Drive/Face_Mask/testing/without_mask/"

split_files(source_mask_normalized,training_with_mask,testing_with_mask,0.8)
split_files(source_without_mask_normalized,training_without_mask,testing_without_mask,0.8)


100%|██████████| 690/690 [00:00<00:00, 4412.75it/s]

100%|██████████| 686/686 [00:00<00:00, 3035.34it/s]


In [19]:
print(len(os.listdir(training_with_mask)))
print(len(os.listdir(training_without_mask)))
print(len(os.listdir(testing_with_mask)))
print(len(os.listdir(testing_without_mask)))

552
548
138
138


**Checkpoint for storing best model**


In [35]:
#Defining a new directory for storing checkpoint

#os.mkdir('/content/drive/My Drive/Face_Mask/model_weights/')

EPOCHS = 5
checkpoint_filepath = '/content/drive/My Drive/Face_Mask/model_weights/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)


# Defining the model

In [36]:
def model(Training_Generator,Validation_Generator):
  mask = tf.keras.models.Sequential([
                             tf.keras.layers.Conv2D(200,(3,3),activation=tf.nn.relu,input_shape = (150,150,3)),
                             tf.keras.layers.MaxPooling2D(2,2),
                             
                             tf.keras.layers.Conv2D(100,(3,3),activation=tf.nn.relu),
                             tf.keras.layers.MaxPooling2D(2,2),
                              
                             tf.keras.layers.Conv2D(50,(3,3),activation=tf.nn.relu),
                             tf.keras.layers.MaxPooling2D(2,2),
                              
                              
                             tf.keras.layers.Conv2D(50,(3,3),activation=tf.nn.relu),
                             tf.keras.layers.MaxPooling2D(2,2),
                              
                             tf.keras.layers.Flatten(), 
                             tf.keras.layers.Dense(50,activation =tf.nn.relu),
                             tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)                
                            

  ])
  mask.summary()
  mask.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
  history = mask.fit_generator( Training_Generator,
                              epochs = 20,
                              verbose = 2,
                              validation_data = Validation_Generator,
                              callbacks = [model_checkpoint_callback] )
                              
  

**Creating the Training and Validation Data**

In [23]:
training_dir = "/content/drive/My Drive/Face_Mask/training/"
train_datagen = ImageDataGenerator(rescale = 1/255,
                                   rotation_range =135,
                                   vertical_flip=True,
                                   shear_range =0.2,
                                   height_shift_range =0.5,
                                   width_shift_range =0.5,
                                   fill_mode='nearest')

validation_dir ='/content/drive/My Drive/Face_Mask/testing/'
validation_datagen = ImageDataGenerator(rescale = 1/255,
                                   rotation_range =135,
                                   vertical_flip=True,
                                   shear_range =0.2,
                                   height_shift_range =0.5,
                                   width_shift_range =0.5,
                                   fill_mode='nearest')
training_generator = train_datagen.flow_from_directory(
    training_dir,
    batch_size = 10,
    target_size =(150,150),
    class_mode ='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    batch_size = 10,
    target_size =(150,150),
    class_mode ='binary'
)

Found 1100 images belonging to 2 classes.
Found 276 images belonging to 2 classes.


# Calling the model

In [37]:
mask = model(training_generator,validation_generator)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 148, 148, 200)     5600      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 74, 74, 200)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 72, 72, 100)       180100    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 36, 36, 100)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 34, 34, 50)        45050     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 17, 17, 50)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 15, 15, 50)       

NameError: ignored